In [43]:
# standard python imports
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import copy

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [44]:
data = np.load('exp_io_data\CoupledPendula_mean_in7_out7_Tmax2.0_data.npz')
for key, val in data.items():
    print(key)
    exec(key +'=val')

xlist
exp_out_list


In [45]:
x=xlist
y=exp_out_list
#print(y)

In [46]:
print(x.shape)
print(y.shape)

(2000, 14)
(2, 60, 2000, 7, 2)


In [48]:
#y1=y[0:1,29:30,:,:,0:1]
print(y.shape)

#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
#y1_np=torch.from_numpy(y1)
#y1=torch.transpose(y1_np,1,2)
#print(y1.shape)
y2=y[0:1,:,0:1000,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y2_np=torch.from_numpy(y2)
y2=torch.transpose(y2_np,1,2)
print(y2.shape)
#print(y1.shape)

(2, 60, 2000, 7, 2)
torch.Size([1, 1000, 60, 7, 1])


NameError: name 'y1' is not defined

In [49]:
y3=y2.squeeze(0).squeeze(3)
print(y3.shape)

torch.Size([1000, 60, 7])


In [59]:
y_train=y3[:,1:2,3:4]
print(y_train.shape)

torch.Size([1000, 1, 1])


In [60]:
x_train=x[0:1000,:]
print(x_train.shape)
print(x)

(1000, 14)
[[0.9515082  0.02859658 0.36347437 ... 0.35507327 0.8292224  0.4047569 ]
 [0.25135255 0.7703726  0.775273   ... 0.14982301 0.15127295 0.7362834 ]
 [0.19514024 0.16856277 0.07611024 ... 0.7874241  0.22519547 0.09913057]
 ...
 [0.47233897 0.5813042  0.5970881  ... 0.4770273  0.8321323  0.0551641 ]
 [0.9792367  0.9304237  0.7407218  ... 0.07495439 0.42210817 0.49420857]
 [0.77542037 0.6565631  0.83288056 ... 0.03630865 0.04533863 0.08364367]]


In [61]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(14, 20 ,bias=True)
        self.layer2 = nn.Linear(20, 10,bias=True)
        self.layer3 = nn.Linear(10, 1,bias=True)


    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #x = F.relu(self.layer4(x))
        #x = F.relu(self.layer5(x))
        #x = torch.sigmoid(self.layer6(x))
        return x


net = Net()
print(net)

Net(
  (layer1): Linear(in_features=14, out_features=20, bias=True)
  (layer2): Linear(in_features=20, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
)


In [62]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

6
torch.Size([20, 14])


In [63]:
x_data=torch.tensor(x_train)
input = x_data
out = net(input)
print(out)
print(out.shape)

tensor([[-3.4358e-02],
        [ 9.7913e-02],
        [-7.2161e-03],
        [ 1.2311e-01],
        [ 9.2735e-02],
        [-2.0208e-01],
        [-1.5921e-01],
        [ 5.6794e-02],
        [-4.6093e-02],
        [-1.6022e-01],
        [-2.0517e-02],
        [ 5.3976e-02],
        [-4.1024e-02],
        [ 1.2974e-01],
        [-1.2074e-01],
        [-8.9911e-04],
        [ 1.8922e-02],
        [ 1.7253e-01],
        [ 5.8241e-02],
        [ 1.8916e-01],
        [ 9.3307e-02],
        [ 3.9296e-02],
        [-8.7348e-02],
        [-6.6106e-02],
        [ 1.1119e-01],
        [-9.9904e-02],
        [ 5.4341e-02],
        [ 7.9236e-02],
        [ 4.4595e-02],
        [-8.2010e-02],
        [ 7.1116e-02],
        [ 4.4362e-02],
        [-4.2856e-02],
        [-3.2425e-02],
        [ 1.3633e-01],
        [-4.7816e-02],
        [ 1.5615e-01],
        [ 8.2684e-02],
        [ 6.8190e-02],
        [-3.9882e-02],
        [ 1.1449e-01],
        [ 2.4433e-02],
        [-4.1025e-02],
        [-6

In [64]:
net.zero_grad()
out.backward(torch.randn(1000,1))

In [65]:
target=y_train
target_new=target.view(1000,1)
print(target_new.shape)
criterion=nn.MSELoss()
loss=criterion(out,target_new)
print(loss)

torch.Size([1000, 1])
tensor(1.1705, grad_fn=<MseLossBackward>)


In [66]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.01, betas=(0.9, 0.999), eps=1e-09, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)


tensor(1.1705, grad_fn=<MseLossBackward>)
tensor(1.1272, grad_fn=<MseLossBackward>)
tensor(1.0907, grad_fn=<MseLossBackward>)
tensor(1.0588, grad_fn=<MseLossBackward>)
tensor(1.0297, grad_fn=<MseLossBackward>)
tensor(1.0010, grad_fn=<MseLossBackward>)
tensor(0.9713, grad_fn=<MseLossBackward>)
tensor(0.9401, grad_fn=<MseLossBackward>)
tensor(0.9068, grad_fn=<MseLossBackward>)
tensor(0.8705, grad_fn=<MseLossBackward>)
tensor(0.8309, grad_fn=<MseLossBackward>)
tensor(0.7877, grad_fn=<MseLossBackward>)
tensor(0.7402, grad_fn=<MseLossBackward>)
tensor(0.6888, grad_fn=<MseLossBackward>)
tensor(0.6337, grad_fn=<MseLossBackward>)
tensor(0.5754, grad_fn=<MseLossBackward>)
tensor(0.5135, grad_fn=<MseLossBackward>)
tensor(0.4495, grad_fn=<MseLossBackward>)
tensor(0.3847, grad_fn=<MseLossBackward>)
tensor(0.3192, grad_fn=<MseLossBackward>)
tensor(0.2559, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1423, grad_fn=<MseLossBackward>)
tensor(0.0965, grad_fn=<MseLossBac

In [81]:
x_test=x[1010:1020,:]
print(x_test.shape)
y1=y[0:1,:,1010:1020,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y1_np=torch.from_numpy(y1)
y1=torch.transpose(y1_np,1,2)

(10, 14)


In [82]:
y4=y1.squeeze(0).squeeze(3)
print(y4.shape)

y_test=y4[:,1:2,3:4]
print(y_test.shape)

torch.Size([10, 60, 7])
torch.Size([10, 1, 1])


In [83]:
x_test=torch.tensor(x_test)
test_out=net(x_test)
#LOSS = criterion(test_out, y_test)
print(test_out)
print(y_test)

tensor([[-0.4104],
        [-1.2037],
        [ 0.4397],
        [-1.1533],
        [-1.4568],
        [ 1.5126],
        [ 0.4783],
        [-1.4773],
        [-1.4551],
        [-0.5478]], grad_fn=<AddmmBackward>)
tensor([[[-0.5427]],

        [[-1.4222]],

        [[ 0.5540]],

        [[-1.4101]],

        [[-1.7900]],

        [[ 1.7833]],

        [[ 0.5466]],

        [[-1.7075]],

        [[-1.7122]],

        [[-0.6049]]])


In [ ]:

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()

        self.layer1 = nn.Linear(14, 120)
        self.layer2 = nn.Linear(120, 120)
        self.layer3 = nn.Linear(120, 120)
        self.layer4 = nn.Linear(120, 84)
        self.layer5 = nn.Linear(84, 42)
        self.layer6 = nn.Linear(42, 7)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.layer1(x))
        x = x+F.relu(self.layer2(x))
        x = x+F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.tanh(self.layer5(x))
        x = F.relu(self.layer6(x))
        return x


net2 = Net2()
print(net2)

In [ ]:
params2 = list(net2.parameters())
print(len(params2))
print(params2[0].size())

In [ ]:
x_data=torch.tensor(x)
input = x_data
out = net2(input)
print(out)
print(out.shape)

In [ ]:
target=y1[0:1000,:,:]
target_new=target.view(1000,7)

In [ ]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net2(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)
